<a href="https://colab.research.google.com/github/E1250/courses_ref/blob/main/Secure%20and%20Private%20AI/Section%202%20-%20Federated%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line.

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [11]:
# !pip install syft
# !pip install --upgrade --force-reinstall zstd

In [12]:
import torch as th

In [13]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [14]:
y = x + x

In [15]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [16]:
import syft as sy

In [17]:
hook = sy.TorchHook(th) # Modify PyTorch with new Functionality

AttributeError: ignored

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x = x.send(bob)

In [ ]:
bob._objects

{50844634909: tensor([1, 2, 3, 4, 5])}

In [ ]:
x.location

<VirtualWorker id:bob #tensors:1>

In [ ]:
x.id_at_location

50844634909

In [ ]:
x.id

85673375777

In [ ]:
x.owner

<VirtualWorker id:me #tensors:0>

In [ ]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [ ]:
x

(Wrapper)>[PointerTensor | me:85673375777 -> bob:50844634909]

In [ ]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [ ]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
alice = sy.VirtualWorker(hook, id="alice")

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x_ptr = x.send(bob, alice)

In [ ]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [ ]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [ ]:
x

(Wrapper)>[PointerTensor | me:7344279461 -> bob:74976771769]

In [ ]:
y

(Wrapper)>[PointerTensor | me:68830170639 -> bob:70981483926]

In [ ]:
z = x + y

In [ ]:
z

(Wrapper)>[PointerTensor | me:79875157913 -> bob:79875157913]

In [ ]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [ ]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:28437210120 -> bob:28437210120]

In [ ]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [ ]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()

In [ ]:
z.backward()

In [ ]:
x = x.get()

In [ ]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [ ]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [ ]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [ ]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [ ]:
bob = bob.clear_objects()

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [ ]:
del x

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = "asdf"

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [ ]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = "asdf"

In [ ]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [ ]:
del x

In [ ]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [ ]:
bob = bob.clear_objects()
bob._objects

{}

In [ ]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{17426510898: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [ ]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [ ]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)

train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)

    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [ ]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [ ]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [ ]:
y = x + x

In [ ]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [ ]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [ ]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [ ]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [ ]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{}

In [ ]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [ ]:
bob._objects

{}

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [ ]:
del x

In [ ]:
bob._objects

{}

In [ ]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [ ]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{}

In [ ]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [ ]:
bob._objects

{}

In [ ]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [ ]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [ ]:
bob._objects

{}

In [ ]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [ ]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [ ]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [ ]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}